# Lab 2: Multi-Agent Systems with AutoGen and CrewAI

**Objective:** Build multi-agent systems using two different frameworks and compare their approaches.

**Duration:** ~45 minutes

**What You'll Learn:**
- How to build conversational multi-agent systems with AutoGen
- How to create role-based agent crews with CrewAI
- Trade-offs between different multi-agent approaches

## Part 1: Setup

First, let's set up our environment for both frameworks.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv()

# Verify API key
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found"
print("Environment configured!")

## Part 2: Introduction to AutoGen

AutoGen uses a conversational approach where agents communicate through messages. Key concepts:

- **AssistantAgent**: An AI-powered agent that can use tools and respond
- **UserProxyAgent**: Represents the user, can execute code
- **GroupChat**: Enables multiple agents to converse

In [ ]:
# AutoGen imports
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

print("AutoGen imported successfully!")

In [ ]:
# Configure the LLM
llm_config = {
    "config_list": [{
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    }],
    "temperature": 0.7
}

print("LLM configuration ready!")

### Building a Simple Two-Agent System

Let's start with a simple assistant and user proxy.

In [ ]:
# Create an assistant agent
assistant = AssistantAgent(
    name="Assistant",
    system_message="""You are a helpful AI assistant. 
    Provide clear, concise answers to questions.
    When you've fully answered a question, end with 'TERMINATE'.""",
    llm_config=llm_config
)

# Create a user proxy (simulates user interaction)
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  # Automatic mode
    max_consecutive_auto_reply=3,
    code_execution_config=False,  # Disable code execution for safety
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "")
)

print("Basic agents created!")

In [ ]:
# Test the simple interaction
user_proxy.initiate_chat(
    assistant,
    message="What are the main benefits of using AI agents in software development?"
)

### Building a Multi-Agent Research Team

Now let's create a team of specialized agents that collaborate.

In [ ]:
# Research Analyst Agent
researcher = AssistantAgent(
    name="Researcher",
    system_message="""You are a research analyst. Your job is to:
    1. Break down research topics into key questions
    2. Identify important facts and data points
    3. Provide well-sourced information
    
    Focus on accuracy and depth. Cite your reasoning.""",
    llm_config=llm_config
)

# Writer Agent
writer = AssistantAgent(
    name="Writer",
    system_message="""You are a professional writer. Your job is to:
    1. Take research findings and turn them into clear, engaging content
    2. Structure information logically
    3. Use accessible language
    
    Wait for the Researcher to provide information before writing.""",
    llm_config=llm_config
)

# Critic Agent
critic = AssistantAgent(
    name="Critic",
    system_message="""You are a constructive critic. Your job is to:
    1. Review content for accuracy and clarity
    2. Identify gaps or weaknesses
    3. Suggest specific improvements
    
    Be constructive and specific in your feedback.
    When the content is satisfactory, say 'APPROVED' and 'TERMINATE'.""",
    llm_config=llm_config
)

print("Research team agents created!")

In [ ]:
# Create a group chat
group_chat = GroupChat(
    agents=[user_proxy, researcher, writer, critic],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin"  # Agents take turns
)

# Create a manager to orchestrate the chat
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

print("Group chat configured!")

In [ ]:
# Run the multi-agent research task
user_proxy.initiate_chat(
    manager,
    message="""Write a brief article about the benefits and challenges of 
    remote work for software developers. Keep it under 300 words."""
)

## Part 3: Introduction to CrewAI

CrewAI uses a role-based approach where you define:

- **Agents**: With specific roles, goals, and backstories
- **Tasks**: Specific work items with expected outputs
- **Crew**: The team that executes tasks

In [ ]:
# CrewAI imports
from crewai import Agent, Task, Crew, Process

print("CrewAI imported successfully!")

### Building a CrewAI Research Team

Let's build the same research team using CrewAI's approach.

In [ ]:
# Research Analyst Agent
crew_researcher = Agent(
    role="Research Analyst",
    goal="Conduct thorough research and provide accurate, well-sourced information",
    backstory="""You are an experienced research analyst with expertise in 
    gathering and analyzing information. You pride yourself on accuracy and 
    depth of research. You always break down complex topics into key questions.""",
    verbose=True,
    allow_delegation=False
)

# Writer Agent
crew_writer = Agent(
    role="Content Writer",
    goal="Transform research into clear, engaging, and well-structured content",
    backstory="""You are a skilled content writer with a talent for making 
    complex topics accessible. You create engaging narratives while maintaining 
    accuracy. You always structure your content logically.""",
    verbose=True,
    allow_delegation=False
)

# Critic Agent  
crew_critic = Agent(
    role="Content Reviewer",
    goal="Ensure content quality through constructive feedback and improvements",
    backstory="""You are a meticulous editor with years of experience reviewing 
    content. You have a keen eye for inconsistencies, unclear passages, and 
    areas that need improvement. You provide specific, actionable feedback.""",
    verbose=True,
    allow_delegation=False
)

print("CrewAI agents created!")

In [ ]:
# Define tasks
research_task = Task(
    description="""Research the topic: Benefits and challenges of remote work 
    for software developers.
    
    Identify:
    - 3 key benefits with supporting points
    - 3 main challenges with context
    - Any relevant statistics or trends""",
    expected_output="A structured research summary with key findings",
    agent=crew_researcher
)

writing_task = Task(
    description="""Using the research findings, write a brief article about 
    remote work for software developers.
    
    Requirements:
    - Under 300 words
    - Clear structure with introduction and conclusion
    - Engaging but professional tone""",
    expected_output="A polished article under 300 words",
    agent=crew_writer,
    context=[research_task]  # This task depends on research
)

review_task = Task(
    description="""Review the article for:
    - Accuracy of information
    - Clarity and readability
    - Structure and flow
    
    Provide the final approved version with any necessary corrections.""",
    expected_output="The final, reviewed article ready for publication",
    agent=crew_critic,
    context=[writing_task]  # This task depends on writing
)

print("Tasks defined!")

In [ ]:
# Create the crew
research_crew = Crew(
    agents=[crew_researcher, crew_writer, crew_critic],
    tasks=[research_task, writing_task, review_task],
    process=Process.sequential,  # Tasks run in order
    verbose=True
)

print("Crew assembled!")

In [ ]:
# Run the crew
result = research_crew.kickoff()

print("\n" + "=" * 50)
print("FINAL OUTPUT:")
print("=" * 50)
print(result)

## Part 4: Framework Comparison

Let's compare the two approaches.

In [ ]:
comparison = {
    "AutoGen": {
        "approach": "Conversational message-passing",
        "strengths": [
            "Natural conversation flow between agents",
            "Built-in code execution capabilities",
            "Flexible speaker selection methods",
            "Good for dynamic, exploratory tasks"
        ],
        "weaknesses": [
            "Can be harder to control conversation direction",
            "May require more iterations to reach goal",
            "Less structured output"
        ],
        "best_for": "Open-ended discussions, code generation, human-in-the-loop"
    },
    "CrewAI": {
        "approach": "Role-based task execution",
        "strengths": [
            "Clear task definitions and dependencies",
            "Intuitive role/goal/backstory pattern",
            "Structured, predictable workflow",
            "Easy to understand and modify"
        ],
        "weaknesses": [
            "Less flexible for dynamic tasks",
            "Agents don't naturally 'discuss'",
            "Sequential by default"
        ],
        "best_for": "Defined workflows, content creation, structured pipelines"
    }
}

print("Framework Comparison")
print("=" * 60)
for framework, details in comparison.items():
    print(f"\n{framework}")
    print(f"  Approach: {details['approach']}")
    print(f"  Best for: {details['best_for']}")
    print(f"  Strengths:")
    for s in details['strengths']:
        print(f"    + {s}")
    print(f"  Weaknesses:")
    for w in details['weaknesses']:
        print(f"    - {w}")

## Challenge: Build Your Own Multi-Agent System

Choose one of the following scenarios and implement it using **either** AutoGen or CrewAI:

### Option A: Code Review Team
- **Code Reviewer**: Analyzes code for bugs and issues
- **Security Expert**: Checks for security vulnerabilities
- **Optimizer**: Suggests performance improvements

### Option B: Product Planning Team
- **Product Manager**: Defines requirements
- **Developer**: Assesses technical feasibility
- **Designer**: Considers user experience

### Option C: Customer Support Team
- **First Responder**: Initial customer contact
- **Technical Support**: Deep technical issues
- **Escalation Manager**: Handles complex cases

In [ ]:
# TODO: Implement your chosen multi-agent system

# If using AutoGen:
# agent1 = AssistantAgent(...)
# agent2 = AssistantAgent(...)
# agent3 = AssistantAgent(...)
# group_chat = GroupChat(...)

# If using CrewAI:
# agent1 = Agent(...)
# agent2 = Agent(...)
# agent3 = Agent(...)
# task1 = Task(...)
# task2 = Task(...)
# crew = Crew(...)

In [ ]:
# TODO: Test your multi-agent system
# Run it with a sample input and observe the collaboration

In [ ]:
# TODO: Document your observations
my_observations = {
    "framework_chosen": "",  # AutoGen or CrewAI
    "why_this_framework": "",
    "what_worked_well": "",
    "challenges_faced": "",
    "would_change": ""
}

print("Document your observations above!")

## Lab Summary

In this lab, you learned:

1. **AutoGen Basics**: Creating `AssistantAgent`, `UserProxyAgent`, and `GroupChat`
2. **CrewAI Basics**: Defining `Agent`, `Task`, and `Crew`
3. **Multi-Agent Patterns**: How agents collaborate in different frameworks
4. **Trade-offs**: When to choose each framework

### Key Takeaways

- **AutoGen** excels at dynamic, conversational workflows
- **CrewAI** excels at structured, role-based workflows
- Good agent definitions (roles, goals, system messages) are crucial
- Multi-agent systems can produce better results than single agents
- Choose the framework that matches your workflow pattern

### Decision Guide

| Scenario | Recommended Framework |
|----------|----------------------|
| Open-ended exploration | AutoGen |
| Code generation with execution | AutoGen |
| Human-in-the-loop | AutoGen |
| Defined content pipeline | CrewAI |
| Clear role separation | CrewAI |
| Structured deliverables | CrewAI |

### Next Steps

In Session 2, you'll learn about:
- LangGraph for stateful workflows
- Conditional routing and cycles
- Building your capstone project

In [ ]:
print("Lab 2 complete!")
print("\nYou've now experienced two different approaches to multi-agent systems.")
print("Remember: There's no 'best' framework - choose based on your use case!")